In [1]:
using Rx
using BenchmarkTools

┌ Info: Precompiling Rx [df971d30-c9d6-4b37-b8ff-e965b2cb3a40]
└ @ Base loading.jl:1273


In [2]:
n = 500

array      = collect(1:n)
observable = from(array);

In [3]:
reactive(source::S) where S = reactive(as_subscribable(S), source)

function reactive(::ValidSubscribable{T}, source) where T
    keep = KeepActor{T}()
    subscribe!(source, keep)
    return keep.values
end

reactive (generic function with 2 methods)

# Map operator performance comparison

In [4]:
function regularArrayMap(source)
    keep = Vector{Int}()
    for v in source
        push!(keep, v ^ 2)
    end
    return keep
end

@CreateMapOperator(SquaredInt, Int, Int, (d) -> d ^ 2)

observable_mapped1 = observable |> SquaredIntMapOperator()
observable_mapped2 = observable |> map(Int, (d) -> d ^ 2)

println(regularArrayMap(array) == reactive(observable_mapped1))
println(regularArrayMap(array) == reactive(observable_mapped2))

@btime regularArrayMap($array);
@btime reactive($observable_mapped1);
@btime reactive($observable_mapped2);

true
true
  3.174 μs (9 allocations: 8.33 KiB)
  3.489 μs (11 allocations: 8.36 KiB)
  25.780 μs (489 allocations: 15.84 KiB)


# Filter operator performance comparison

In [5]:
function regularArrayFilter(source)
    keep   = Vector{Int}()
    for v in source
        if v % 2 == 0
            push!(keep, v)
        end
    end
    return keep
end

@CreateFilterOperator(EvenInt, Int, (d) -> d % 2 == 0)

observable_filtered1 = observable |> EvenIntFilterOperator()
observable_filtered2 = observable |> filter((d) -> d % 2 == 0)

println(regularArrayFilter(array) == reactive(observable_filtered1))
println(regularArrayFilter(array) == reactive(observable_filtered2))

@btime regularArrayFilter($array);
@btime reactive($observable_filtered1);
@btime reactive($observable_filtered2);

true
true
  1.879 μs (8 allocations: 4.27 KiB)
  2.146 μs (10 allocations: 4.30 KiB)
  10.537 μs (10 allocations: 4.31 KiB)


# Count operator performance comparison

In [6]:
function regularArrayCount(source)
    keep    = Vector{Int}()
    current = 0
    for v in source
        current += 1
    end
    push!(keep, current)
    return keep
end

observable_counted = observable |> count()

println(regularArrayCount(array) == reactive(observable_counted))

@btime regularArrayCount($array);
@btime reactive($observable_counted);

true
  52.204 ns (2 allocations: 128 bytes)
  278.517 ns (4 allocations: 176 bytes)


# Enumerate operator performance comparison

In [7]:
function regularArrayEnumerate(source)
    keep   = Vector{Tuple{Int, Int}}()
    current = 1
    for v in source
        push!(keep, (v, current))
        current += 1
    end
    return keep
end

observable_enumerated = observable |> enumerate()

println(regularArrayEnumerate(array) == reactive(observable_enumerated))

@btime regularArrayEnumerate($array);
@btime reactive($observable_enumerated);

true
  3.751 μs (9 allocations: 16.34 KiB)
  4.143 μs (11 allocations: 16.39 KiB)


# Scan operator performance comparison

In [8]:
function regularArrayScan(source)
    keep    = Vector{Int}()
    current = 0
    for v in source
        current = current + v
        push!(keep, current)
    end
    return keep
end

@CreateScanOperator(Sum, Int, Int, +)

observable_scanned1 = observable |> SumScanOperator(0)
observable_scanned2 = observable |> scan(Int, (d, c) -> d + c, 0)

println(regularArrayScan(array) == reactive(observable_scanned1))
println(regularArrayScan(array) == reactive(observable_scanned2))

@btime regularArrayScan($array);
@btime reactive($observable_scanned1);
@btime reactive($observable_scanned2);

true
true
  3.196 μs (9 allocations: 8.33 KiB)
  3.672 μs (11 allocations: 8.38 KiB)
  149.297 μs (1886 allocations: 37.69 KiB)


# Reduce operator performance comparison

In [9]:
function regularArrayReduce(source)
    keep    = Vector{Int}()
    current = 0
    for v in source
        current = current + v
    end
    push!(keep, current)
    return keep
end

@CreateReduceOperator(Sum, Int, Int, +)

observable_reduced1 = observable |> SumReduceOperator(0)
observable_reduced2 = observable |> reduce(Int, (d, c) -> d + c, 0)

println(regularArrayReduce(array) == reactive(observable_reduced1))
println(regularArrayReduce(array) == reactive(observable_reduced2))

@btime regularArrayReduce($array);
@btime reactive($observable_reduced1);
@btime reactive($observable_reduced2);

true
true
  83.195 ns (2 allocations: 128 bytes)
  308.516 ns (4 allocations: 176 bytes)
  140.905 μs (1879 allocations: 29.48 KiB)


# Max operator performance comparison

In [10]:
function regularArrayMax(source)
    keep    = Vector{Int}()
    current = nothing
    for v in source
        if current == nothing
            current = v
        else
            if v > current
                current = v
            end
        end
    end
    push!(keep, current)
    return keep
end

observable_max = observable |> max()

println(regularArrayMax(array) == reactive(observable_max))

@btime regularArrayMax($array);
@btime reactive($observable_max);

true
  551.053 ns (2 allocations: 128 bytes)
  1.027 μs (4 allocations: 176 bytes)


# Min operator performance comparison

In [11]:
function regularArrayMin(source)
    keep    = Vector{Int}()
    current = nothing
    for v in source
        if current == nothing
            current = v
        else
            if v < current
                current = v
            end
        end
    end
    push!(keep, current)
    return keep
end

observable_min = observable |> min()

println(regularArrayMin(array) == reactive(observable_min))

@btime regularArrayMin($array);
@btime reactive($observable_min);

true
  1.036 μs (2 allocations: 128 bytes)
  1.036 μs (4 allocations: 176 bytes)


# Sum operator performance comparison

In [12]:
function regularArraySum(source)
    keep    = Vector{Int}()
    current = nothing
    for v in source
        if current == nothing
            current = v
        else
            current = current + v
        end
    end
    push!(keep, current)
    return keep
end

observable_sum = observable |> sum()

println(regularArraySum(array) == reactive(observable_sum))

@btime regularArraySum($array);
@btime reactive($observable_sum);

true
  667.203 ns (2 allocations: 128 bytes)
  895.778 ns (4 allocations: 176 bytes)
